In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sqlite3
import random
from datetime import datetime, timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:


np.random.seed(42)

# Define the number of samples
num_samples = 50000


customer_ids = np.arange(num_samples)


ages = np.random.randint(18, 71, size=num_samples)

#Gender
genders = np.random.choice(['Male', 'Female', 'Other'], size=num_samples, p=[0.48, 0.48, 0.04])


marital_statuses = np.random.choice(['Single', 'Married', 'Divorced', 'Widowed', 'Separated'], size=num_samples, p=[0.45, 0.4, 0.1, 0.02, 0.03])

#Education Level
education_levels = np.random.choice(['High School', 'Associate Degree', 'Bachelor Degree', 'Master Degree', 'Doctorate'], size=num_samples, p=[0.3, 0.2, 0.3, 0.15, 0.05])

#Geographic Information
states = ['Florida', 'Michigan', 'New York', 'Virginia', 'Ohio', 'Illinois', 'California', 'Oregon', 'Washington', 'Pennsylvania']
geographic_info = np.random.choice(states, size=num_samples)

#Occupation
occupations = ['Entrepreneur', 'Manager', 'Salesperson', 'Engineer', 'Teacher', 'Healthcare', 'IT Professional', 'Clerk', 'Technician', 'Consultant']
occupation = np.random.choice(occupations, size=num_samples)

#Income Level
income_level = np.random.normal(loc=60000, scale=20000, size=num_samples).astype(int)
income_level = np.clip(income_level, 20000, 150000)

#Behavioral Data
behavioral_data = np.random.normal(loc=30, scale=10, size=num_samples).astype(int)
behavioral_data = np.clip(behavioral_data, 5, 120)

#Purchase History
start_date = datetime(2019, 12, 1)
end_date = datetime(2023, 12, 1)

#Generate random dates between dates
purchase_history = [start_date + timedelta(days=random.randint(0, (end_date - start_date).days)) for _ in range(num_samples)]

#Insurance Products Owned
insurance_products = np.random.randint(1, 6, size=num_samples)

#Policy Type
policy_types = ['Individual', 'Family', 'Group']
policy_type = np.random.choice(policy_types, size=num_samples, p=[0.5, 0.3, 0.2])

#Customer Preferences
customer_preferences = np.random.randint(1, 6, size=num_samples)

#Preferred Communication Channel
communication_channels = ['Email', 'Phone', 'Mail', 'Text', 'Mobile App', 'Chat']
preferred_communication = np.random.choice(communication_channels, size=num_samples)

#Preferred Contact Time
contact_times = ['Morning', 'Afternoon', 'Evening', 'Anytime', 'Weekends']
preferred_contact_time = np.random.choice(contact_times, size=num_samples)

#Preferred Language
languages = ['English', 'French', 'German', 'Spanish', 'Hindi', 'Mandarin']
preferred_language = np.random.choice(languages, size=num_samples)

#Segmentation Group
segmentation_groups = ['Segment1', 'Segment2', 'Segment3', 'Segment4', 'Segment5']
segmentation_group = np.random.choice(segmentation_groups, size=num_samples)

# Create df
data = pd.DataFrame({
    'Customer ID': customer_ids,
    'Age': ages,
    'Gender': genders,
    'Marital Status': marital_statuses,
    'Education Level': education_levels,
    'Geographic Information': geographic_info,
    'Occupation': occupation,
    'Income Level': income_level,
    'Behavioral Data': behavioral_data,
    'Purchase History': purchase_history,
    'Insurance Products Owned': insurance_products,
    'Policy Type': policy_type,
    'Customer Preferences': customer_preferences,
    'Preferred Communication Channel': preferred_communication,
    'Preferred Contact Time': preferred_contact_time,
    'Preferred Language': preferred_language,
    'Segmentation Group': segmentation_group
})

#Map Insurance Products Owned
policy_names = ['policy1', 'policy2', 'policy3', 'policy4', 'policy5']
mapped_policy_names = [policy_names[i-1] for i in insurance_products]
data['Insurance Products Owned'] = mapped_policy_names

# 20. Map Insurance Products Owned to the values [1, 3, 2, 5, 4] for calculations
insurance_mapping = {1: 1, 2: 3, 3: 2, 4: 5, 5: 4}
mapped_insurance_products = [insurance_mapping[i] for i in insurance_products]

# 21. Calculate Coverage Amount using mapped values
coverage_amount = (
    500 * data['Income Level'] / 1000 +
    1000 * data['Age'] +
    20000 * np.array(mapped_insurance_products) +
    np.random.normal(0, 10000, size=num_samples)  # Adding noise
)

# 22. Calculate Premium Amount
premium_amount = (
    0.05 * coverage_amount +
    3000 * data['Education Level'].map({'High School': 1, 'Associate Degree': 2, 'Bachelor Degree': 3, 'Master Degree': 4, 'Doctorate': 5}) +
    1000 * data['Customer Preferences'] +
    np.random.normal(0, 5000, size=num_samples)  # Adding noise
)

# Assign to DataFrame
data['Coverage Amount'] = coverage_amount.astype(int)
data['Premium Amount'] = premium_amount.astype(int)

# Display the first few rows
df = data
data


In [ ]:
conn = sqlite3.connect('customerdba')
cursor = conn.cursor()

#Customers Table
cursor.execute('''
DROP TABLE IF EXISTS Customers;
''')
cursor.execute('''
DROP TABLE IF EXISTS CustomerPolicies;
''')
cursor.execute('''
DROP TABLE IF EXISTS CustomerPreferences;
''')
cursor.execute('''
DROP TABLE IF EXISTS InsuranceProducts;
''')
cursor.execute('''
CREATE TABLE Customers (
    CustomerID INTEGER PRIMARY KEY,
    Age INTEGER,
    Gender TEXT,
    MaritalStatus TEXT,
    EducationLevel TEXT,
    GeographicInformation TEXT,
    Occupation TEXT,
    IncomeLevel REAL,
    BehavioralData TEXT
);
''')

#CustomerPolicies Table
cursor.execute('''
CREATE TABLE CustomerPolicies (
    PolicyID INTEGER PRIMARY KEY AUTOINCREMENT,
    CustomerID INTEGER,
    PurchaseHistory TEXT,
    InsuranceProductsOwned TEXT,
    CoverageAmount REAL,
    PremiumAmount REAL,
    PolicyType TEXT,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (InsuranceProductsOwned) REFERENCES InsuranceProducts(ProductName)
);
''')

#CustomerPreferences Table
cursor.execute('''
CREATE TABLE CustomerPreferences (
    PreferenceID INTEGER PRIMARY KEY AUTOINCREMENT,
    CustomerID INTEGER,
    CustomerPreferences1 INTEGER,
    PreferredCommunicationChannel TEXT,
    PreferredContactTime TEXT,
    PreferredLanguage TEXT,
    SegmentationGroup TEXT,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
''')

#InsuranceProducts Table
cursor.execute('''
CREATE TABLE InsuranceProducts (
    ProductName TEXT PRIMARY KEY,
    ProductDescription TEXT
);
''')

conn.commit()

In [ ]:
df['Coverage Amount'] = pd.to_numeric(df['Coverage Amount'])
df['Premium Amount'] = pd.to_numeric(df['Premium Amount'])
df['Purchase History'] = pd.to_datetime(df['Purchase History'], errors='coerce').dt.strftime('%Y-%m-%d')

In [ ]:
for _, row in df.iterrows():
    # Insert into Customers table
    cursor.execute('''
    INSERT INTO Customers (CustomerID, Age, Gender, MaritalStatus, EducationLevel, GeographicInformation, Occupation, IncomeLevel, BehavioralData)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        row['Customer ID'], row['Age'], row['Gender'], row['Marital Status'], row['Education Level'],
        row['Geographic Information'], row['Occupation'], float(row['Income Level']), row['Behavioral Data']
    ))

    # Insert into InsuranceProducts table (if needed) before inserting into CustomerPolicies
    # Ensure InsuranceProductsOwned in CustomerPolicies references ProductName
    insurance_products = {
        'policy1': 'Comprehensive health insurance covering a wide range of medical needs',
        'policy2': 'Affordable health insurance with flexible coverage options',
        'policy3': 'Premium health insurance offering extensive benefits and services',
        'policy4': 'Reliable health insurance designed for long-term healthcare security',
        'policy5': 'Specialized health insurance tailored to meet unique individual needs'
    }

    # Check and insert insurance product based on row['Insurance Products Owned']
    if row['Insurance Products Owned'] in insurance_products:
        product_name = row['Insurance Products Owned']
        product_description = insurance_products[product_name]

        cursor.execute('''
        INSERT OR IGNORE INTO InsuranceProducts (ProductName, ProductDescription)
        VALUES (?, ?)
        ''', (product_name, product_description))

    # Insert into CustomerPolicies table
    cursor.execute('''
    INSERT INTO CustomerPolicies (CustomerID, PurchaseHistory, InsuranceProductsOwned, CoverageAmount, PremiumAmount, PolicyType)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        row['Customer ID'], row['Purchase History'], row['Insurance Products Owned'],
        row['Coverage Amount'], row['Premium Amount'], row['Policy Type']
    ))

    # Insert into CustomerPreferences table
    cursor.execute('''
    INSERT INTO CustomerPreferences (CustomerID, CustomerPreferences1, PreferredCommunicationChannel, PreferredContactTime, PreferredLanguage, SegmentationGroup)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        row['Customer ID'], row['Customer Preferences'], row['Preferred Communication Channel'], row['Preferred Contact Time'],
        row['Preferred Language'], row['Segmentation Group']
    ))
# Commit the changes and close the connection
conn.commit()

In [ ]:
incomeGrouping = """
SELECT
    CustomerID, Age, Gender, IncomeLevel,MaritalStatus,EducationLevel,
    CASE
        WHEN IncomeLevel > (SELECT AVG(IncomeLevel) FROM Customers) THEN 'Above Average'
        ELSE 'Below Average'
    END AS Income_Group
FROM Customers;"""

iG = pd.read_sql_query(incomeGrouping, conn)
iG

In [ ]:
highValueCustomers = """
SELECT c.Age,c.Gender,c.MaritalStatus,c.Occupation,c.IncomeLevel,cp.PolicyType,cp.CoverageAmount,cp.PremiumAmount,p.PreferredCommunicationChannel,p.PreferredLanguage
FROM
    Customers c
JOIN
    CustomerPolicies cp ON c.CustomerID = cp.CustomerID
JOIN
    CustomerPreferences p ON c.CustomerID = p.CustomerID
ORDER BY
    cp.CoverageAmount DESC, c.IncomeLevel DESC
LIMIT 10;"""
hVC = pd.read_sql_query(highValueCustomers, conn)
hVC

In [ ]:
sg = """SELECT p.SegmentationGroup, cp.PolicyType, COUNT(cp.PolicyID) AS policyCount
FROM CustomerPreferences p
JOIN CustomerPolicies cp ON p.CustomerID = cp.CustomerID
GROUP BY p.SegmentationGroup, cp.PolicyType"""
sg = pd.read_sql_query(sg, conn)
sg

In [ ]:
query = '''
SELECT
    c.CustomerID AS "Customer ID",
    c.Age,
    c.Gender,
    c.MaritalStatus AS "Marital Status",
    c.EducationLevel AS "Education Level",
    c.GeographicInformation AS "Geographic Information",
    c.Occupation,
    c.IncomeLevel AS "Income Level",
    c.BehavioralData AS "Behavioral Data",
    cp.PurchaseHistory AS "Purchase History",
    cp.InsuranceProductsOwned AS "Insurance Products Owned",
    cp.CoverageAmount AS "Coverage Amount",
    cp.PremiumAmount AS "Premium Amount",
    cp.PolicyType AS "Policy Type",
    cr.CustomerPreferences1 AS "Customer Preferences",
    cr.PreferredCommunicationChannel AS "Preferred Communication Channel",
    cr.PreferredContactTime AS "Preferred Contact Time",
    cr.PreferredLanguage AS "Preferred Language",
    cr.SegmentationGroup AS "Segmentation Group"
FROM
    Customers c
LEFT JOIN
    CustomerPolicies cp ON c.CustomerID = cp.CustomerID
LEFT JOIN
    CustomerPreferences cr ON c.CustomerID = cr.CustomerID;
'''


df1 = pd.read_sql_query(query, conn)
df1

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
age_distribution= df['Age'].value_counts().sort_index()
plt.figure(figsize=(12,6))
age_distribution.plot(kind='bar', color='green')
plt.title('Age distribution of customers')
plt.xlabel('Age')
plt.ylabel('number of customers')
plt.show()

In [ ]:
gender_distribution=df['Gender'].value_counts().sort_index()
plt.figure(figsize=(12,6))
gender_distribution.plot(kind='bar', color='blue')
plt.title('Gender distribution of customers')
plt.xlabel('Gender')
plt.ylabel('number fo customers')
plt.show()

In [ ]:
marital_status_distribution=df['Marital Status'].value_counts().sort_index()
plt.figure(figsize=(12,6))
marital_status_distribution.plot(kind='bar', color='black')
plt.title('Marital Status distribution of customers')
plt.xlabel('Marital Status')
plt.ylabel('number fo customers')
plt.show()

In [ ]:
education_distribution=df['Education Level'].value_counts().sort_index()
plt.figure(figsize=(12,6))
education_distribution.plot(kind='bar', color='red')
plt.title('Educaiton level distribution of customers')
plt.xlabel('Education level')
plt.ylabel('Number of customers')
plt.show()

In [ ]:
geo_distribution=df['Geographic Information'].value_counts().sort_index()
plt.figure(figsize=(12,6))
geo_distribution.plot(kind='bar')
plt.title('Geographic distribution of customers')
plt.xlabel('Geographic information')
plt.ylabel('number of customers')
plt.show()

In [ ]:
age_distribution= df['Occupation'].value_counts().sort_index()
plt.figure(figsize=(12,6))
age_distribution.plot(kind='bar', color='green')
plt.title('Occupation distribution of customers')
plt.xlabel('Occupation')
plt.ylabel('number of customers')
plt.show()

In [ ]:
age_distribution= df['Behavioral Data'].value_counts().sort_index()
plt.figure(figsize=(12,6))
age_distribution.plot(kind='bar', color='yellow')
plt.title('Behavioral Data distribution of customers')
plt.xlabel('Behavioral Data')
plt.ylabel('number of customers')
plt.show()

In [ ]:
numerical_columns = ['Age', 'Income Level', 'Behavioral Data', 'Coverage Amount', 'Premium Amount']
plt.figure(figsize=(10, 8))
sns.heatmap(df[numerical_columns].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age', y='Premium Amount', hue='Gender', data=df)
plt.title("Age vs Premium Amount by Gender")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age', y='Coverage Amount', hue='Gender', data=df)
plt.title("Age vs Coverage Amount by Gender")
plt.show()

In [ ]:
# Conclusion

# In summary, this exploratory data analysis (EDA) provided insights into the customer segmentation dataset.
# Visualizations revealed the distribution of key demographic and behavioral characteristics, including age, gender, marital status, education level, geographic location, occupation, and behavioral data.
# These visualizations highlight potential customer segments based on these characteristics, laying the groundwork for more advanced segmentation techniques like clustering or classification algorithms.
# Further analysis could focus on correlations between these features and customer spending habits, product preferences, or other relevant metrics to refine segmentation strategies and improve targeted marketing efforts.
# The identified customer segments can inform business decisions around product development, marketing campaigns, and customer relationship management.

In [ ]:
clean_df = df.copy()

# Ensure 'Purchase History' is in datetime format and convert it to number of days
clean_df['Purchase History'] = pd.to_datetime(clean_df['Purchase History'], errors='coerce')
start_date = datetime(2019, 12, 1)
clean_df['Purchase History'] = (clean_df['Purchase History'] - start_date).dt.days

# Features and Targets
X = clean_df.drop(['Customer ID', 'Coverage Amount', 'Premium Amount'], axis=1)
y_coverage = clean_df['Coverage Amount']
y_premium = clean_df['Premium Amount']

# One-Hot Encoding for Categorical Variables
X_encoded = pd.get_dummies(X, drop_first=True)

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

In [ ]:
# Coverage Amount
X_train_cov, X_test_cov, y_train_cov, y_test_cov = train_test_split(
    X_scaled, y_coverage, test_size=0.2, random_state=42
)

# Premium Amount
X_train_prem, X_test_prem, y_train_prem, y_test_prem = train_test_split(
    X_scaled, y_premium, test_size=0.2, random_state=42
)

In [ ]:
# Initialize models
linear_reg_cov = LinearRegression()
random_forest_cov = RandomForestRegressor(random_state=42, n_estimators=100)

linear_reg_prem = LinearRegression()
random_forest_prem = RandomForestRegressor(random_state=42, n_estimators=100)

# Train models for Coverage Amount
linear_reg_cov.fit(X_train_cov, y_train_cov)
random_forest_cov.fit(X_train_cov, y_train_cov)

# Train models for Premium Amount
linear_reg_prem.fit(X_train_prem, y_train_prem)
random_forest_prem.fit(X_train_prem, y_train_prem)

In [ ]:

# Coverage Amount Prediction
print("\n--- Coverage Amount Prediction ---")
y_pred_cov_lr = linear_reg_cov.predict(X_test_cov)
mse_cov_lr = mean_squared_error(y_test_cov, y_pred_cov_lr)
r2_cov_lr = r2_score(y_test_cov, y_pred_cov_lr)
print(f"Linear Regression - MSE: {mse_cov_lr:.2f}, R²: {r2_cov_lr:.4f}")

y_pred_cov_rf = random_forest_cov.predict(X_test_cov)
mse_cov_rf = mean_squared_error(y_test_cov, y_pred_cov_rf)
r2_cov_rf = r2_score(y_test_cov, y_pred_cov_rf)
print(f"Random Forest - MSE: {mse_cov_rf:.2f}, R²: {r2_cov_rf:.4f}")

# Storing results
results_cov = {
    "Linear Regression": {"MSE": mse_cov_lr, "R²": r2_cov_lr},
    "Random Forest": {"MSE": mse_cov_rf, "R²": r2_cov_rf},
}

# Premium Amount Prediction
print("\n--- Premium Amount Prediction ---")
y_pred_prem_lr = linear_reg_prem.predict(X_test_prem)
mse_prem_lr = mean_squared_error(y_test_prem, y_pred_prem_lr)
r2_prem_lr = r2_score(y_test_prem, y_pred_prem_lr)
print(f"Linear Regression - MSE: {mse_prem_lr:.2f}, R²: {r2_prem_lr:.4f}")

y_pred_prem_rf = random_forest_prem.predict(X_test_prem)
mse_prem_rf = mean_squared_error(y_test_prem, y_pred_prem_rf)
r2_prem_rf = r2_score(y_test_prem, y_pred_prem_rf)
print(f"Random Forest - MSE: {mse_prem_rf:.2f}, R²: {r2_prem_rf:.4f}")

# Storing results
results_prem = {
    "Linear Regression": {"MSE": mse_prem_lr, "R²": r2_prem_lr},
    "Random Forest": {"MSE": mse_prem_rf, "R²": r2_prem_rf},
}

In [ ]:

# Residual Plot
plt.figure(figsize=(8, 6))
sns.residplot(x=y_test_cov, y=y_pred_cov_rf, lowess=True, line_kws={"color": "red"})
plt.title("Residual Plot  (Coverage Amount)")
plt.xlabel("Actual Values")
plt.ylabel("Residuals")
plt.show()

# Feature Importance for Linear Regression (Coverage Amount)
coef_cov_lr = pd.Series(
    linear_reg_cov.coef_, index=X_encoded.columns
).sort_values(ascending=False)
print('\n')
print("--- Feature Importance (Coefficients) for Linear Regression (Coverage Amount) ---")
print(coef_cov_lr.head(10))

# Feature Importance for Random Forest
rf_cov = RandomForestRegressor(random_state=42, n_estimators=100)
rf_cov.fit(X_train_cov, y_train_cov)
importances_cov = pd.Series(rf_cov.feature_importances_, index=X_encoded.columns)
print('\n')
print("--- Feature Importance for Coverage Amount ---")
print(importances_cov.sort_values(ascending=False).head(10))

In [ ]:
# Use the predictions from Random Forest for residual plot
y_pred_prem = y_pred_prem_rf  # Replace with y_pred_prem_lr if you want Linear Regression predictions

# Residual Plot
plt.figure(figsize=(8, 6))
sns.residplot(x=y_test_prem, y=y_pred_prem, lowess=True, line_kws={"color": "red", "lw": 2})
plt.title("Residual Plot (Premium Amount)")
plt.xlabel("Actual Values")
plt.ylabel("Residuals")
plt.show()

# Feature Importance for Linear Regression (Premium Amount)
coef_prem_lr = pd.Series(
    linear_reg_prem.coef_, index=X_encoded.columns
).sort_values(ascending=False)

print('\n')
print("--- Feature Importance (Coefficients) for Linear Regression (Premium Amount) ---")
print(coef_prem_lr.head(10))

# Feature Importance for Random Forest
rf_prem = RandomForestRegressor(random_state=42, n_estimators=100)
rf_prem.fit(X_train_prem, y_train_prem)
importances_prem = pd.Series(rf_prem.feature_importances_, index=X_encoded.columns)
print('\n')
print("--- Feature Importance for Premium Amount ---")
print(importances_prem.sort_values(ascending=False).head(10))